In [ ]:
# installing requirements, uncomment and run if needed!
#!pip -q install ./python

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name="/data/Tennis_Linux/Tennis.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


In [ ]:
INPUT_SIZE = 48         # state size
NUM_ACTIONS = 4         # the number of possible actions
SEED = 13               # a random seed for reproducibility

In [ ]:
import torch
from ddpg_agent import Agent

#create the agent
agent = Agent(state_size=INPUT_SIZE, action_size=4, random_seed=SEED)
agent.actor_local.load_state_dict(torch.load('trained_actor.pth'))
agent.actor_local.eval()
print(agent.actor_local)
agent.critic_local.load_state_dict(torch.load('trained_critic.pth'))
agent.critic_local.eval()
print(agent.critic_local)

In [ ]:
for i in range(10):
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = env_info.vector_observations.flatten()            # get the current state
    score = 0                                          # initialize the score
    while True:
        #just here, expand state to shape (1,33) for our bn
        action = agent.act(np.expand_dims(state.flatten(),axis=0))
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations.flatten()   # get the next state
        reward = max(env_info.rewards)                   # get the reward
        done = max(env_info.local_done)                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Episode: {} Score: {}".format(i, score))

In [ ]:
env.close()